# Episode state evolution

Notebook that evalutes the solarMED model for some date, and plots the evolution of the states during the episode.

In [ ]:
from pathlib import Path
from loguru import logger
import hjson
from solarmed_modeling.utils import data_preprocessing, data_conditioning
from solarmed_modeling.solar_med import (ModelParameters, 
                                         FixedModelParameters, 
                                         FsmParameters,
                                         EnvironmentParameters)
from solarmed_modeling.solar_med.utils import evaluate_model
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

%load_ext autoreload
%autoreload 2

logger.disable("phd_visualizations.utils")
logger.enable("solarmed_modeling.solar_med.utils")
logger.enable("solarmed_modeling.utils")

data_path: Path = Path("../../data")
src_diagram_path = Path('../../auxiliar_material/solarMED_optimization-Operating modes.svg')

date_str: str = "20230703" # '20230630'
filename_process_data = f'{date_str}_solarMED.csv'
filename_process_data2 = f'{date_str}_MED.csv'

sample_rate = '400s'
sample_rate_numeric = int(sample_rate[:-1])

# Resample figures using plotly_resampler
resample_figures: bool = False


### Pre-processing

In [2]:
output_path: Path = Path(date_str)
output_path.mkdir(exist_ok=True)


In [3]:
cost_params = EnvironmentParameters()
data_paths = [data_path / f"datasets/{filename_process_data}", data_path / f"datasets/{filename_process_data2}"]

with open( data_path / "variables_config.hjson") as f:
    vars_config = hjson.load(f)
    
with open(data_path/"plot_config_validation.hjson") as f:
    plot_config = hjson.load(f)
    
# Load data and preprocess data
df = data_preprocessing(data_paths, vars_config, sample_rate_key=sample_rate)

# Condition data
df = data_conditioning(df, sample_rate_numeric=sample_rate_numeric, vars_config=vars_config, cost_w=cost_params.cost_w, cost_e=cost_params.cost_e)


2025-08-31 17:24:49.071 | INFO     | solarmed_modeling.utils:data_preprocessing:113 - Reading data from 20230703_solarMED.csv


2025-08-31 17:24:49.253 | DEBUG    | solarmed_modeling.utils:process_dataframe:69 - Index([], dtype='object')
2025-08-31 17:24:49.255 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 0
2025-08-31 17:24:49.255 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 0
2025-08-31 17:24:49.275 | INFO     | solarmed_modeling.utils:data_preprocessing:133 - Reading data from 20230703_MED.csv
2025-08-31 17:24:49.623 | DEBUG    | solarmed_modeling.utils:process_dataframe:69 - Index([], dtype='object')
2025-08-31 17:24:49.625 | INFO     | solarmed_modeling.utils:process_dataframe:76 - Number of duplicate index values in df: 0
2025-08-31 17:24:49.629 | DEBUG    | solarmed_modeling.utils:data_preprocessing:152 - Common columns in both dataframes: Index(['FT-AQU-100', 'TT-AQU-107a', 'CT-DES-001', 'FT-DES-003', 'FT-DES-005',
       'SW2TC1', 'TE-DES-015', 'TT-DES-030'],
      dtype='object'), dropping them

In [4]:
dfs_mod, stats = evaluate_model(
    df=df, sample_rate = sample_rate_numeric,
    model_params = ModelParameters(), 
    fixed_model_params = FixedModelParameters(),
    fsm_params=FsmParameters(),
    env_params=EnvironmentParameters(),
    alternatives_to_eval = ["constant-water-props"],
)

# fig = experimental_results_plot(plot_config, df, df_comp=dfs_mod[0], vars_config=vars_config, resample=resample_figures)

# fig.show(
#     config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
# )


2025-08-31 17:24:49.902 | INFO     | solarmed_modeling.solar_med.utils:evaluate_model:66 - Starting evaluation of alternative constant-water-props. Sample rate = 400 s


 /workspaces/SolarMED/modeling/src/solarmed_modeling/metrics.py:167: RuntimeWarning:invalid value encountered in divide
2025-08-31 17:24:51.367 | INFO     | solarmed_modeling.solar_med.utils:evaluate_model:154 - Finished evaluation of alternative constant-water-props. Elapsed time: 1.459 s, MAE: 12.79 ºC


## Episode state evolution visualization

In [8]:
from solarmed_modeling.visualization.fsm.state_evolution import plot_episode_state_evolution
from solarmed_modeling.fsms import SfTsState, MedState

df_mod = dfs_mod[0]

fig = plot_episode_state_evolution(df_mod, subsystems_state_cls=[SfTsState, MedState], show_edges=False, show_inputs_subplot=True)

fig


FigureWidget({
    'data': [{'hoverinfo': 'name+x+y',
              'legendgroup': 'inputs',
              'name': 'sf_active',
              'showlegend': True,
              'stackgroup': 'inputs',
              'type': 'scatter',
              'uid': '6a10c397-5b7a-470f-81a8-37644e005d6b',
              'x': {'bdata': 'AAQIDBAUGBwgJCgsMDQ4PEBESExQVFhcYA==', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAAAAAAAAAAADwPwAAAAAAAP' ... 'AAAADwPwAAAAAAAPA/AAAAAAAAAAA='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'name+x+y',
              'legendgroup': 'inputs',
              'name': 'ts_active',
              'showlegend': True,
              'stackgroup': 'inputs',
              'type': 'scatter',
              'uid': '15ac06f3-ead8-4a4c-a0b7-df3b137f09ba',
              'x': {'bdata': 'AAQIDBAUGBwgJCgsMDQ4PEBESExQVFhcYA==', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('AAA

In [10]:
from phd_visualizations import save_figure

save_figure(
    fig,
    figure_path=Path("/workspaces/SolarMED/modeling/results"),
    figure_name=f"solarmed_fsm_evolution_{date_str}",
    formats=["png"]
)


2025-08-31 17:33:29.190 | INFO     | phd_visualizations:save_figure:41 - Figure saved in /workspaces/SolarMED/modeling/results/solarmed_fsm_evolution_20230703.png


### Episode state evolution with facility diagram animation

In [10]:
# Hihglight state evolution to generate a GIF
from cairosvg import svg2png
from copy import deepcopy
from lxml import etree
from phd_visualizations.utils import stack_images_vertically
from solarmed_modeling.visualization.fsm.state_evolution import plot_episode_state_evolution
from solarmed_modeling.visualization.fsm.facility_diagram import SolarMedStateVisualizer
from solarmed_modeling.fsms import SfTsState, MedState

logger.disable("phd_visualizations.diagrams")

plot_path = output_path / "plots"
plot_path.mkdir(exist_ok=True)

diagram_path = output_path / "diagrams"
diagram_path.mkdir(exist_ok=True)

height=600
width=1800

with open(src_diagram_path, 'r') as f:
    diagram_file = etree.parse(f)

fig = plot_episode_state_evolution(df_mod, subsystems_state_cls=[SfTsState, MedState], show_edges=False)

for step_idx in fig.layout.xaxis.tickvals:
    # Generate plot
    plot_episode_state_evolution(df_mod, subsystems_state_cls=[SfTsState, MedState], 
                                 show_edges=False, highligth_step=step_idx,
                                 width=width, height=height).write_image(plot_path / f"step_{step_idx:03}.png")
    
    # Generate diagram
    sf_state, ts_state, med_state = df_mod.loc[df_mod.index[step_idx], ['sf_state', 'ts_state', 'med_state']]
    solar_med_state = SolarMedStateVisualizer(sf_state=sf_state.value, ts_state=ts_state.value, med_state=med_state.value)

    diagram = solar_med_state.create_state_diagram(deepcopy(diagram_file))
    diagram.write( diagram_path / f'step_{step_idx:03}.svg')
    
    svg2png(url=str(diagram_path / f'step_{step_idx:03}.svg'), write_to=str(diagram_path / f'step_{step_idx:03}.png'), 
            output_width=width, output_height=height, dpi=300, background_color='white')

    stack_images_vertically(plot_path / f"step_{step_idx:03}.png", diagram_path / f"step_{step_idx:03}.png", output_path / f"step_{step_idx:03}.png")


Generating the GIF requires having installed `imagemagick`

In [ ]:
import subprocess
import os

# Newer systems
# magick *.png -morph 3 -delay 50 -loop 0 output.gif
# Older systems
# convert *.png -morph 3 -delay 50 -loop 0 output.gif

# ffmpeg -framerate 3 -pattern_type glob -i '*.png' output.gif

# Change directory to output_path
os.chdir(output_path)
subprocess.run(["magick", "*.png", "-morph", "3", "-delay", "50", "-loop", "0", "output.gif"], shell=True)
